In [104]:
import numpy as np
from astropy.io import fits

In [48]:
band_names = list('ugriz')

In [40]:
col_defs = [
    fits.Column(name='ID', format='K'),
    fits.Column(name='Tata', format='L'),
]
for b in band_names:
    col_defs.append(fits.Column(name=b, format='E'))
col_defs.append(fits.Column(name='Something Else', format='J'))

In [41]:
table = fits.BinTableHDU.from_columns(col_defs, nrows=100)
data = table.data

In [49]:
dtype2 = np.dtype({name: data.dtype.fields[name] for name in band_names + ['ID']})
dtype2

dtype({'names':['ID','u','g','r','i','z'], 'formats':['<i8','<f4','<f4','<f4','<f4','<f4'], 'offsets':[0,9,13,17,21,25], 'itemsize':29})

In [73]:
dtype2['ID'].itemsize, dtype2.fields['ID'][1]

(8, 0)

In [76]:
dtype2 = np.dtype({name: data.dtype.fields[name] for name in band_names})
dtype2

dtype({'names':['u','g','r','i','z'], 'formats':['<f4','<f4','<f4','<f4','<f4'], 'offsets':[9,13,17,21,25], 'itemsize':29})

In [88]:
x = np.ndarray(data.shape, (np.float32, len(band_names)), data, dtype2.fields['u'][1], data.strides)

In [89]:
x.shape

(100, 5)

In [91]:
np.may_share_memory(x, data)

True

In [92]:
x[:,0] = 1
x[:,4] = 55

In [94]:
x[0:10,:]

array([[ 1.,  0.,  0.,  0., 55.],
       [ 1.,  0.,  0.,  0., 55.],
       [ 1.,  0.,  0.,  0., 55.],
       [ 1.,  0.,  0.,  0., 55.],
       [ 1.,  0.,  0.,  0., 55.],
       [ 1.,  0.,  0.,  0., 55.],
       [ 1.,  0.,  0.,  0., 55.],
       [ 1.,  0.,  0.,  0., 55.],
       [ 1.,  0.,  0.,  0., 55.],
       [ 1.,  0.,  0.,  0., 55.]], dtype=float32)

In [98]:
data['u'][0:10], data['z'][0:10]

(array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32),
 array([55., 55., 55., 55., 55., 55., 55., 55., 55., 55.], dtype=float32))

In [147]:
def flat_view(data, fields):
    selected = [data.dtype.fields[c] for c in fields]
    dtypes = [f[0] for f in selected]
    offsets = np.array([f[1] for f in selected])
    sizes = np.array([data.dtype[f].itemsize for f in fields])
    if len(set(dtypes)) > 1:
        raise Exception('Must have the same type!')

    consecutive = (offsets[:-1] + sizes[:-1] == offsets[1:]).all()
    if not consecutive:
        raise Exception('Must be consecutive!')

    view = np.ndarray(data.shape, dtype=(dtypes[0], len(dtypes)), buffer=data, offset=offsets[0], strides=data.strides)
    assert np.may_share_memory(view, data)
    return view

In [148]:
y = flat_view(data, band_names)

In [152]:
y[:,1] = 22

In [163]:
data.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False